# ANÁLISIS PREDICTIVO DEL ESTADO DE LOS EMBALSES Y ESTUDIO DE SU EVOLUCIÓN HISTÓRICA

Este notebook explica el trabajo de preprocesamiento y preparación de los datos para implementar una serie de visualizaciónes que **expliquen el estado y la evolución temporal de la reserva hídrica del país para los embalses con capacidad superior a 5hm3**. Además, para este conjunto de datos se va a realizar un **análisis predictivo de series temporales**. Para ello realizaremos un análisis exploratorio de los datos, definiremos y validaremos el modelo predictivo, para por último generar las predicciones junto a sus gráficas y visualizaciones correspondientes.

\
Para llevar a cabo esta tarea de manera práctica hemos escogido conjuntos de datos que contienen información relevante sobre los embalses a largo de los últimos años. A partir de estos datos observaremos las características principales y las comparativas en sus evoluciones. Aunque se definirán con mayor detalle en el informe, las fuentes de información principales han sido:

* [Ministerio para la Transición Ecológica y el Reto Demográfico](https://www.miteco.gob.es/es/)
* [Catálogo de Metadatos](https://www.mapama.gob.es/ide/metadatos/srv/spa/catalog.search#/home)
* [Boletín Hidrológico Semanal](https://www.miteco.gob.es/es/agua/temas/evaluacion-de-los-recursos-hidricos/boletin-hidrologico.html)



## 1.Importación de librerías a utilizar

Lo primero que debemos hacer es instalar y cargar las librerías para el preprocesamiento de los datos. Las librerías principales que se van a utilizar para la realización de este proyecto son las siguientes:



* [pandas](https://pandas.pydata.org/) es una librería de Python especializada en el manejo y análisis de estructuras de datos.
* [numpy](https://numpy.org) es el paquete fundamental para el cálculo numérico en Python.
* [datetime](https://docs.python.org/es/3/library/datetime.html) es el módulo que permite trabajar con datos de fechas y manipular su formato
* [matplotlib](https://matplotlib.org/) es una librería de Python especializada en la creación de visualizaciones.
* [seaborn](https://seaborn.pydata.org/) es una librería de Python basada en matplotlib especializada en la creación de visualizaciones.
* [xgboost](https://xgboost.readthedocs.io/en/stable/) es una librería de Python para la implementación de algoritmos de machine learning en el marco de "Gradient Boosting".



In [21]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

## 2.Carga de datos

Realizamos la carga de los conjuntos de datos para explorarlos y realizar algunas tareas básicas de limpieza y procesado de datos. Importaremos para ello la función que nos permite conectar nuestro notebook al almacenamiento de **Google Drive**, ya que se ha creado una carpeta en la que se encuentran los conjuntos de datos (raw_data).

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
embalses_path = "/content/drive/MyDrive/Colab Notebooks/CEI_project/raw_data/Datos_embalses_1998-2024_5hm3.xlsx"
df_embalses = pd.read_excel(embalses_path)

Visualizamos la información principal del dataset generado con la información de los embalses. El objetivo es realizar un primer vistazo a la información contenida en este dataset. Repetiremos el mismo proceso con los datos de geolozalización de los embales (presas).

In [24]:
df_embalses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678585 entries, 0 to 678584
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   AMBITO_NOMBRE   678585 non-null  object        
 1   EMBALSE_NOMBRE  678585 non-null  object        
 2   FECHA           678585 non-null  datetime64[ns]
 3   AGUA_TOTAL      678583 non-null  object        
 4   AGUA_ACTUAL     678583 non-null  object        
 5   ELECTRICO_FLAG  678585 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 31.1+ MB


In [25]:
df_embalses.head()

,AMBITO_NOMBRE,EMBALSE_NOMBRE,FECHA,AGUA_TOTAL,AGUA_ACTUAL,ELECTRICO_FLAG
0,Miño - Sil,Albarellos,1988-01-05,"91,00","32,00",1
1,Miño - Sil,Albarellos,1988-01-12,"91,00","44,00",1
2,Miño - Sil,Albarellos,1988-01-19,"91,00","42,00",1
3,Miño - Sil,Albarellos,1988-01-26,"91,00","43,00",1
4,Miño - Sil,Albarellos,1988-02-02,"91,00","65,00",1


In [26]:
embalses_geo_path = "/content/drive/MyDrive/Colab Notebooks/CEI_project/raw_data/geo_data_embalses5hm3.xlsx"
df_embalses_geo = pd.read_excel(embalses_geo_path)

In [27]:
df_embalses_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X                          367 non-null    float64
 1   Y                          367 non-null    float64
 2   INFORME                    367 non-null    object 
 3   ID_INFRAES                 367 non-null    int64  
 4   CODIGO                     367 non-null    int64  
 5   NOMBRE                     367 non-null    object 
 6   EMBALSE                    367 non-null    object 
 7   Wikidata                   187 non-null    object 
 8   Google Knowledge Graph ID  115 non-null    object 
 9   Google                     115 non-null    object 
 10  OpenStreetMap relation ID  36 non-null     float64
 11  OpenStreetMap              36 non-null     object 
 12  image                      157 non-null    object 
 13  Imagen                     157 non-null    object 

In [28]:
df_embalses_geo.head()

,X,Y,INFORME,ID_INFRAES,CODIGO,NOMBRE,EMBALSE,Wikidata,Google Knowledge Graph ID,Google,...,USO,SUP_CUENCA,AP_M_ANUAL,NMN_CAPAC,NMN_SUP,COTA_CORON,ALT_CIMIEN,LONG_CORON,CAP_AL_NAE,CAP_MAX_DF
0,36.153615,-5.649731,https://sig.mapama.gob.es/WebServices/clientew...,1364,5110039,ALMODOVAR,Embalse de Almodóvar,https://www.wikidata.org/wiki/Q5369429,NaN,NaN,...,Abastecimiento\nDefensa frente avenidas,16.5,0.00,5.000,650200.0,107.2,40.0,126.0,0.0,NaN
1,42.722873,-0.462684,https://sig.mapama.gob.es/WebServices/clientew...,2482,9220067,IBON DE IP,Ip reservoir,https://www.wikidata.org/wiki/Q23999337,NaN,NaN,...,Hidroeléctrico,8.0,4.56,5.000,270000.0,2119.0,31.0,158.0,0.0,20.00
2,38.095562,-3.831098,https://sig.mapama.gob.es/WebServices/clientew...,1543,5230007,ZOCUECA,ZOCUECA,NaN,NaN,NaN,...,Abastecimiento\nDefensa frente avenidas,NaN,NaN,5.000,480000.0,NaN,NaN,NaN,0.0,NaN
3,43.277118,-1.773747,https://sig.mapama.gob.es/WebServices/clientew...,84,1310009,SAN ANTON,SAN ANTON,NaN,NaN,NaN,...,Abastecimiento,10.5,20.00,5.091,278170.0,247.0,56.8,210.0,0.0,150.48
4,38.060263,-1.487653,https://sig.mapama.gob.es/WebServices/clientew...,2057,7300021,"CIERVA, LA",Embalse de la Cierva,https://www.wikidata.org/wiki/Q107738426,NaN,NaN,...,Abastecimiento\nDefensa frente avenidas,NaN,NaN,5.130,432000.0,NaN,NaN,NaN,0.0,366000.00


## 3.Explicación de variables/campos del dataset

### 3.1 Dataset información embalses

A continuación, se va a realizar un listado de las variables que aparecen en el dataset `df_embalses` en el que se explicará el significado de cada una de ellas y a qué hacen referencia:
* `AMBITO_NOMBRE`: hace referncia a la confederación hidrográfica a la cual pertenece el embalse. Las Confederaciones Hidrográficas son los organismos encargados de la gestión de las cuencas hidrográficas.
* `EMBALSE_NOMBRE`: hace referncia al nombre del embalse.
* `FECHA`: hace referencia al día/mes/año en el que se registró el volumen de agua del embalse en cuestión.
* `AGUA_TOTAL`: hace referencia a la capacidad total de agua en **hm3** que es capaz de almacenar dicho embalse.
* `AGUA_ACTUAL`: hace referencia al volumen de agua en **hm3** que se ha registrado en una fecha determinada.

### 3.2 Dataset información geográfica de embalses

A continuación, se va a realizar un listado de las variables que aparecen en el dataset `df_embalses_geo`. En este caso, como veremos más adelante, **hay ciertas variables que aparecen en nuestro dataset y que no son relevantes** para nuestro caso de estudio y que, por tanto no se van a analizar. Las **variables que sí intervienen** para este proyecto son:
* `X`: coordenada x para la localización del embalse.
* `Y`: coordenada y para la localización del embalse.
* `NOMBRE`: hace referencia al nombre del embalse.
* `DEMARC`: hace referncia a la confederación hidrográfica a la cual pertenece el embalse.
* `CAUCE`: hace referencia al río el cual ha visto interrumpido su curso para la creación del embalse en cuestión.
* `CCAA`: Comunidad Autónoma en la que se encuentra localizado el embalse.
* `PROVINCIA`: Provincia en la que se encuentra localizado el embalse.

## 4.Modificación y ajuste de las variables

Una vez verificadas las variables que intervienen en el proyecto, vamos a modificarlas buscando que se adecuen a nuestras futuras tablas de trabajo. En este caso, renombramos , modificamos el tipo , eliminamos las que no nos interesan para el análisis y sustituimos los valores decimales con coma por punto.

### 4.1 Dataset información embalses

In [29]:
df_embalses = df_embalses.rename(columns= {"EMBALSE_NOMBRE":"nombre_embalse", "AMBITO_NOMBRE": "demarcacion_hidrografica",
                                           "AGUA_TOTAL": "capacidad_total", "AGUA_ACTUAL": "volumen_actual"})
df_embalses = df_embalses.rename(columns=str.lower)
df_embalses = df_embalses[['nombre_embalse', 'demarcacion_hidrografica', 'capacidad_total', 'fecha', 'volumen_actual']]

In [30]:
df_embalses["capacidad_total"] = df_embalses["capacidad_total"].str.replace(",",".").astype(float)
df_embalses["volumen_actual"] = df_embalses["volumen_actual"].str.replace(",",".").astype(float)
df_embalses["fecha"] = pd.to_datetime(df_embalses["fecha"], format = "%d/%m/%Y")

In [31]:
df_embalses.head()

,nombre_embalse,demarcacion_hidrografica,capacidad_total,fecha,volumen_actual
0,Albarellos,Miño - Sil,91.0,1988-01-05,32.0
1,Albarellos,Miño - Sil,91.0,1988-01-12,44.0
2,Albarellos,Miño - Sil,91.0,1988-01-19,42.0
3,Albarellos,Miño - Sil,91.0,1988-01-26,43.0
4,Albarellos,Miño - Sil,91.0,1988-02-02,65.0


In [34]:
df_embalses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678585 entries, 0 to 678584
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   nombre_embalse            678585 non-null  object        
 1   demarcacion_hidrografica  678585 non-null  object        
 2   capacidad_total           678583 non-null  float64       
 3   fecha                     678585 non-null  datetime64[ns]
 4   volumen_actual            678583 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 25.9+ MB


### 4.2 Dataset información geográfica de embalses

In [35]:
df_embalses_geo = df_embalses_geo[["X","Y","NOMBRE","DEMARC","CAUCE","CCAA", "PROVINCIA"]]

df_embalses_geo = df_embalses_geo.rename(columns= {"NOMBRE":"nombre_embalse", "DEMARC": "demarcacion_hidrografica"})
df_embalses_geo = df_embalses_geo.rename(columns=str.lower)

In [36]:
df_embalses_geo.head()

,x,y,nombre_embalse,demarcacion_hidrografica,cauce,ccaa,provincia
0,36.153615,-5.649731,ALMODOVAR,GUADALETE Y BARBATE,RÍO ALMODÓVAR,Andalucía,Cádiz
1,42.722873,-0.462684,IBON DE IP,EBRO,BARRANCO DE IP,Aragón,Huesca
2,38.095562,-3.831098,ZOCUECA,GUADALQUIVIR,RÍO RUMBLAR,Andalucía,Jaén
3,43.277118,-1.773747,SAN ANTON,CANTABRICO ORIENTAL,RIO ENDARA,Comunidad Foral de Navarra,Navarra
4,38.060263,-1.487653,"CIERVA, LA",SEGURA,RIO MULA,Región de Murcia,Murcia
